In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import layers
import tensorflow as tf
IMAGE_ORDERING = 'channels_last'
tf.executing_eagerly()
BATCH_SIZE=32

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block,
               strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with
    strides=(2,2) and the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), data_format=IMAGE_ORDERING, strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, data_format=IMAGE_ORDERING,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
               name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
                      strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x
def one_side_pad(x):
    x = ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING)(x)
    if IMAGE_ORDERING == 'channels_first':
        x = Lambda(lambda x: x[:, :, :-1, :-1])(x)
    elif IMAGE_ORDERING == 'channels_last':
        x = Lambda(lambda x: x[:, :-1, :-1, :])(x)
    return x
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    
    filters1, filters2, filters3 = filters

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), data_format=IMAGE_ORDERING,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, data_format=IMAGE_ORDERING,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
               name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

def regression_block(input_tensor,o1_tensor):
    
    y = LandslideArea(1024)(input_tensor,o1_tensor)
    y = LandslideArea(512)(y,o1_tensor)
    y = LandslideArea(256)(y,o1_tensor)
    y = LandslideArea(128)(y,o1_tensor)
    y = LandslideArea(64)(y,o1_tensor)
    y = LandslideArea(32)(y,o1_tensor)
    y = LandslideArea(16)(y,o1_tensor)
    y = LandslideArea(1)(y,o1_tensor)
    return y
    
def get_resnet50_unet(n_classes=2,input_height=128,  input_width=128, channels=11):

    assert input_height % 32 == 0
    assert input_width % 32 == 0
    
    if IMAGE_ORDERING == 'channels_first':
        img_input = Input(shape=(channels, input_height, input_width))
    elif IMAGE_ORDERING == 'channels_last':
        img_input = Input(shape=(input_height, input_width, channels))

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    if IMAGE_ORDERING == 'channels_first':
        MERGE_AXIS = 1
    elif IMAGE_ORDERING == 'channels_last':
        MERGE_AXIS = -1
    l1_skip_conn=True
    #x = normalizer()
    x = ZeroPadding2D((3, 3), data_format=IMAGE_ORDERING)(img_input)
    x = Conv2D(64, (4, 4), data_format=IMAGE_ORDERING,
               strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), data_format=IMAGE_ORDERING, strides=(2, 2))(x)
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
    br1=tf.identity(x,name='branch_1')
    br1=AveragePooling2D((2, 2), padding='same')(br1)
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    # classification part
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(x)
    x = (Conv2D(1, (3, 3), padding='valid' , activation='relu' , data_format=IMAGE_ORDERING))(x)
    x = (BatchNormalization())(x)
    o1=Activation(activation="sigmoid",name="sus_output")(x)

    
    model = Model(inputs=[img_input], outputs=[o1])
    return model

In [ ]:
model_cls=get_resnet50_unet(n_classes=1,input_height=128,  input_width=128, channels=13)
model_cls.load_weights('your_file_path') #load weights from the classification model

In [ ]:
def get_regression(n_classes=1,input_height=32,  input_width=32, channels=13):

    assert input_height % 32 == 0
    assert input_width % 32 == 0
    
    if IMAGE_ORDERING == 'channels_first':
        img_input = Input(shape=(channels, input_height, input_width))
    elif IMAGE_ORDERING == 'channels_last':
        img_input = Input(shape=(input_height, input_width, channels))

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    if IMAGE_ORDERING == 'channels_first':
        MERGE_AXIS = 1
    elif IMAGE_ORDERING == 'channels_last':
        MERGE_AXIS = -1
        
    x = ZeroPadding2D((3, 3), data_format=IMAGE_ORDERING)(img_input)
    x = Conv2D(64, (4, 4), data_format=IMAGE_ORDERING,
               strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), data_format=IMAGE_ORDERING, strides=(2, 2))(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_dense1')(x)
    x = MaxPooling2D((3, 3), data_format=IMAGE_ORDERING, strides=(2, 2))(x)
    x=  Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    o1 = Dense(1, activation='relu')(x)
    model = Model(inputs=[img_input], outputs=[o1])
    return model

In [ ]:
model_reg=get_regression()
model_reg.load_weights('your_file_path') #saved model weights from regression part

In [ ]:
def get_combined_model(model_cls,model_reg,n_classes=1,input_height=128,  input_width=128, channels=13):
    assert input_height % 32 == 0
    assert input_width % 32 == 0
    
    if IMAGE_ORDERING == 'channels_first':
        img_input = Input(shape=(channels, input_height, input_width))
    elif IMAGE_ORDERING == 'channels_last':
        img_input = Input(shape=(input_height, input_width, channels))

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    if IMAGE_ORDERING == 'channels_first':
        MERGE_AXIS = 1
    elif IMAGE_ORDERING == 'channels_last':
        MERGE_AXIS = -1
        
    output_sus=model_cls(img_input)
    inputs_rs=tf.reshape(img_input,(-1,32,32,13))
    output_reg=model_reg(inputs_rs)
    
    output_reg_shaped=tf.reshape(output_reg,(-1,4,4,1))
    output_haz=multiply([output_sus,output_reg_shaped])
    
    model = Model(inputs=[img_input], outputs=[output_sus,output_reg_shaped,output_haz])
    return model

In [ ]:
model=get_combined_model(model_cls,model_reg)

In [ ]:

from tqdm.notebook import tqdm
def evaluate_predictions(imgarray,model, patchsize):
    """Extract sample patches of size patchsize x patchsize from an image (imgarray) in a gridwise manner. and predict them then merge again 
    to bigger image
    """
    
    nrows, ncols, nbands = imgarray.shape[-3],imgarray.shape[-2],imgarray.shape[-1]
    total_map_pred = np.zeros((240, 200,3), dtype=np.float32)
    mask=np.zeros((240, 200), dtype=np.float32)
    
    for i in tqdm(range(int(nrows/patchsize))):
        for j in range(int(ncols/patchsize)):
            xmin,xmax=i*patchsize,(i+1)*patchsize
            ymin,ymax=j*patchsize,(j+1)*patchsize
            single_patch = imgarray[ xmin:xmax,ymin:ymax, :]
            single_patch = np.expand_dims(single_patch, axis=0)
            predictions_arg=model.predict(single_patch)
            xmin,xmax,ymin,ymax=int(xmin/32),int(xmax/32),int(ymin/32),int(ymax/32)
            #print(xmin,xmax,ymin,ymax)
            total_map_pred[xmin:xmax,ymin:ymax:,0]=predictions_arg[0].reshape((4,4))
            total_map_pred[xmin:xmax,ymin:ymax:,1]=predictions_arg[1].reshape((4,4))
            total_map_pred[xmin:xmax,ymin:ymax:,2]=predictions_arg[2].reshape((4,4))

    return total_map_pred

In [ ]:
import rasterio
import numpy as np
def predictandsave():
    for i in range(4,12):
        dataset=rasterio.open(f"prediction_confusion/covars_{str(i)}.tif")
        array=dataset.read().swapaxes(0,2)
        output_data=evaluate_predictions(array,model, 128)
        output_data=output_data.swapaxes(2,0)
        save_file=f"prediction_confusion/preds_{str(i)}.tif" ##Give the save name to save predicted data in here 
        with rasterio.Env():
            profile = dataset.profile
            profile.update(
                dtype=rasterio.float32,
                count=3,
                width= output_data.shape[-1], 
                height= output_data.shape[-2],
                transform= dataset.transform*dataset.transform.scale(32,32),
                compress='lzw')
            with rasterio.open(save_file, 'w', **profile) as dst:
                dst.write(output_data.astype(rasterio.float32))

In [ ]:
covars-np.load('your covar file')
predicitons=model.predict(covars,batch_size=64)
np.save('predictions_grid2.npy',predicitons)